### Automatic model selection

Notebook for testing the selected models

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
from CMAPSAuxFunctions import TrainValTensorBoard

from keras.models import model_from_json

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPS import CMAPSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler

Using TensorFlow backend.


### Load cmaps data handler

In [2]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Test CMAPSS

In [6]:
# load json and create model
model_folder = "best_models/cmapss/"

json_file = open(model_folder+'bestModel_global.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_cmapss = model_from_json(loaded_model_json)
# load weights into new model
model_cmapss.load_weights(model_folder+"bestModel_global.h5")
print("Loaded model from disk")

model_cmapss.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 368)               129168    
_________________________________________________________________
dropout_243 (Dropout)        (None, 368)               0         
_________________________________________________________________
dense_699 (Dense)            (None, 368)               135792    
_________________________________________________________________
dense_700 (Dense)            (None, 1)                 369       
Total params: 265,329
Trainable params: 265,329
Non-trainable params: 0
_________________________________________________________________


### Test MNIST

In [5]:
#min_max_scaler = MinMaxScaler()

# load json and create model
model_folder = "best_models/mnist/"

json_file = open(model_folder+'bestModel_global.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_mnist = model_from_json(loaded_model_json)
# load weights into new model
model_mnist.load_weights(model_folder+"bestModel_global.h5")
print("Loaded model from disk")

model_mnist.summary()

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "categorical_crossentropy"
metrics = ["accuracy"]

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 48)                37680     
_________________________________________________________________
dense_318 (Dense)            (None, 192)               9408      
_________________________________________________________________
dense_319 (Dense)            (None, 48)                9264      
_________________________________________________________________
dropout_111 (Dropout)        (None, 48)                0         
_________________________________________________________________
dense_320 (Dense)            (None, 10)                490       
Total params: 56,842
Trainable params: 56,842
Non-trainable params: 0
_________________________________________________________________


### Test CIFAR-10